In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_excel('/content/Cashtags_2017-2023.xlsx')

In [ ]:
# Filter tweets mentioning Prudential
stock_tweets = df[df['tweet'].str.contains('Prudential|$PRU', case=False, na=False)]

#Renaming 'created_at' to 'Date'
stock_tweets.rename(columns={'created_at': 'Date'}, inplace=True)

stock_tweets['Date'] = pd.to_datetime(stock_tweets['Date'])

#Extract date part
stock_tweets['Date'] = stock_tweets['Date'].dt.date

<ipython-input-3-7b022606555b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_tweets.rename(columns={'created_at': 'Date'}, inplace=True)
<ipython-input-3-7b022606555b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_tweets['Date'] = pd.to_datetime(stock_tweets['Date'])
<ipython-input-3-7b022606555b>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/index

In [ ]:
# Dropping unnecesary columns

stock_tweets.drop(['author id','id','lang'], axis=1, inplace=True)

<ipython-input-4-f07e51a1f46c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_tweets.drop(['author id','id','lang'], axis=1, inplace=True)


In [ ]:
import re
import string
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Preprocess tweet text
def preprocess_tweet(tweet):
    # Convert to lowercase
    tweet = tweet.lower()

    # Remove special characters
    tweet = re.sub(r'[^\w\s]', '', tweet)

    # Remove URLs
    tweet = re.sub(r'http\S+', '', tweet)
    tweet = re.sub(r'www\S+', '', tweet)

    # Remove hashtags and user mentions
    tweet = re.sub(r'#\w+', '', tweet)
    tweet = re.sub(r'@\w+', '', tweet)

    # Remove punctuation
    tweet = tweet.translate(str.maketrans('', '', string.punctuation))

    # Tokenization
    tokens = word_tokenize(tweet)

    # Remove stop words
    tokens = [token for token in tokens if token not in stopwords.words('english')]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return ' '.join(tokens)

# Apply preprocessing
stock_tweets['cleaned_tweet'] = [preprocess_tweet(tweet) for tweet in stock_tweets['tweet']]

<ipython-input-6-b465d6e61760>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_tweets['cleaned_tweet'] = [preprocess_tweet(tweet) for tweet in stock_tweets['tweet']]


In [ ]:
#Removing duplicates
stock_tweets.drop_duplicates(subset=['cleaned_tweet'], inplace = True)

<ipython-input-7-e58614a5f446>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_tweets.drop_duplicates(subset=['cleaned_tweet'], inplace = True)


In [ ]:
stock_tweets.drop(columns=['tweet'],inplace= True)

<ipython-input-8-612f487d092f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_tweets.drop(columns=['tweet'],inplace= True)


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load FinBERT
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/533 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [ ]:
# Sentiment analysis function
def get_sentiment_score(tweet):
    inputs = tokenizer(tweet, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    probabilities = torch.nn.functional.softmax(logits, dim=-1).numpy()[0]
    sentiment_labels = ['negative', 'neutral', 'positive']
    sentiment = sentiment_labels[probabilities.argmax()]
    return sentiment, probabilities

# Apply sentiment analysis
stock_tweets['sentiment'], stock_tweets['probabilities'] = zip(*stock_tweets['cleaned_tweet'].apply(get_sentiment_score))

<ipython-input-10-a1c38944b4d6>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_tweets['sentiment'], stock_tweets['probabilities'] = zip(*stock_tweets['cleaned_tweet'].apply(get_sentiment_score))
<ipython-input-10-a1c38944b4d6>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_tweets['sentiment'], stock_tweets['probabilities'] = zip(*stock_tweets['cleaned_tweet'].apply(get_sentiment_score))


In [ ]:
stock_tweets

,Date,cleaned_tweet,sentiment,probabilities
8,2017-01-01,henderson group plc 4519000 stake prudential f...,negative,"[0.99997544, 5.4664685e-07, 2.3947463e-05]"
29,2017-01-02,prudential financial inc pru rating lowered ho...,positive,"[0.0952467, 0.01554909, 0.88920414]"
42,2017-01-02,king luther capital management corp sell 7270 ...,negative,"[0.9975074, 2.816804e-05, 0.0024644989]"
49,2017-01-02,prudentialfinancial inc cut position ag mortga...,negative,"[0.99964833, 1.0774795e-05, 0.00034094535]"
52,2017-01-02,prudentialfinancial inc given buy rating sandl...,neutral,"[9.6195116e-08, 0.99999976, 7.8270084e-08]"
...,...,...,...,...
393764,2023-05-31,pru prudentials cfo quits code conduct investi...,negative,"[0.9883288, 3.8989434e-05, 0.011632206]"
395104,2023-05-31,pru prudentials cfo quits code conduct investi...,negative,"[0.87721485, 8.757461e-05, 0.12269769]"
395750,2023-05-31,prudential plc pru dipped 60 533 trading 11k,negative,"[0.9222773, 1.9386312e-05, 0.07770343]"
395762,2023-05-31,prudential plc pru dipped 675 6 trading 11k,positive,"[0.15712278, 7.653564e-05, 0.84280074]"


In [ ]:
stock_tweets.drop(columns=['cleaned_tweet', 'sentiment'],inplace= True)

<ipython-input-12-64ce33862835>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_tweets.drop(columns=['cleaned_tweet', 'sentiment'],inplace= True)


In [ ]:
stock_tweets

,Date,probabilities
8,2017-01-01,"[0.99997544, 5.4664685e-07, 2.3947463e-05]"
29,2017-01-02,"[0.0952467, 0.01554909, 0.88920414]"
42,2017-01-02,"[0.9975074, 2.816804e-05, 0.0024644989]"
49,2017-01-02,"[0.99964833, 1.0774795e-05, 0.00034094535]"
52,2017-01-02,"[9.6195116e-08, 0.99999976, 7.8270084e-08]"
...,...,...
393764,2023-05-31,"[0.9883288, 3.8989434e-05, 0.011632206]"
395104,2023-05-31,"[0.87721485, 8.757461e-05, 0.12269769]"
395750,2023-05-31,"[0.9222773, 1.9386312e-05, 0.07770343]"
395762,2023-05-31,"[0.15712278, 7.653564e-05, 0.84280074]"


In [ ]:
stock_tweets[['negative', 'neutral', 'positive']] = pd.DataFrame(stock_tweets['probabilities'].tolist(), index=stock_tweets.index)

<ipython-input-14-a165ddfc8a54>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_tweets[['negative', 'neutral', 'positive']] = pd.DataFrame(stock_tweets['probabilities'].tolist(), index=stock_tweets.index)
<ipython-input-14-a165ddfc8a54>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_tweets[['negative', 'neutral', 'positive']] = pd.DataFrame(stock_tweets['probabilities'].tolist(), index=stock_tweets.index)
<ipython-input-14-a165ddfc8a54>:1: SettingWithCopyWarning: 
A value is trying to be set 

In [ ]:
stock_tweets

,Date,probabilities,negative,neutral,positive
8,2017-01-01,"[0.99997544, 5.4664685e-07, 2.3947463e-05]",9.999754e-01,5.466468e-07,2.394746e-05
29,2017-01-02,"[0.0952467, 0.01554909, 0.88920414]",9.524670e-02,1.554909e-02,8.892041e-01
42,2017-01-02,"[0.9975074, 2.816804e-05, 0.0024644989]",9.975074e-01,2.816804e-05,2.464499e-03
49,2017-01-02,"[0.99964833, 1.0774795e-05, 0.00034094535]",9.996483e-01,1.077479e-05,3.409453e-04
52,2017-01-02,"[9.6195116e-08, 0.99999976, 7.8270084e-08]",9.619512e-08,9.999998e-01,7.827008e-08
...,...,...,...,...,...
393764,2023-05-31,"[0.9883288, 3.8989434e-05, 0.011632206]",9.883288e-01,3.898943e-05,1.163221e-02
395104,2023-05-31,"[0.87721485, 8.757461e-05, 0.12269769]",8.772148e-01,8.757461e-05,1.226977e-01
395750,2023-05-31,"[0.9222773, 1.9386312e-05, 0.07770343]",9.222773e-01,1.938631e-05,7.770343e-02
395762,2023-05-31,"[0.15712278, 7.653564e-05, 0.84280074]",1.571228e-01,7.653564e-05,8.428007e-01


In [ ]:
stock_tweets.drop(columns=['probabilities'],inplace= True)

<ipython-input-16-8c8057f4a158>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_tweets.drop(columns=['probabilities'],inplace= True)


In [ ]:
def combined_sentiment_score(row, neutral_weight=0.5):
    positive_score = row['positive']
    negative_score = row['negative']
    neutral_score = row['neutral']

    combined_score = positive_score - negative_score

    return combined_score

In [ ]:
# Apply the function to each row to get the combined sentiment score
stock_tweets['combined_score'] = stock_tweets.apply(combined_sentiment_score, axis=1)

<ipython-input-18-b215842c195f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_tweets['combined_score'] = stock_tweets.apply(combined_sentiment_score, axis=1)


In [ ]:
# Group by 'Date' and calculate the mean for 'negative', 'neutral', and 'positive'
stock_tweets_sentiment = stock_tweets.groupby('Date').agg({
    'negative': 'mean',
    'neutral': 'mean',
    'positive': 'mean',
    'combined_score': 'mean'
}).reset_index()

In [ ]:
stock_tweets_sentiment

,Date,negative,neutral,positive,combined_score
0,2017-01-01,0.999975,5.466468e-07,2.394746e-05,-0.999951
1,2017-01-02,0.523101,2.538970e-01,2.230024e-01,-0.300098
2,2017-01-03,0.849531,1.434441e-01,7.024859e-03,-0.842506
3,2017-01-04,0.807890,1.919270e-01,1.829012e-04,-0.807707
4,2017-01-05,0.749952,4.807805e-06,2.500429e-01,-0.499909
...,...,...,...,...,...
1975,2023-05-25,0.999907,7.565240e-05,1.690256e-05,-0.999891
1976,2023-05-26,0.775664,4.829224e-03,2.195071e-01,-0.556157
1977,2023-05-28,0.999853,7.573832e-06,1.391329e-04,-0.999714
1978,2023-05-30,0.500000,5.000000e-01,4.897860e-07,-0.499999


In [ ]:
stock_tweets_sentiment.to_excel('stock_tweets_sentiment(FinBERT).xlsx', index = False)